### Importing modules

In [674]:
import numpy as np
import matplotlib.pyplot as plt
import os
from enum import Enum
from itertools import permutations, combinations, product
from copy import copy
from icecream import ic
from math import atan
import logging
from functools import cmp_to_key
from tabulate import tabulate
import random
from datetime import datetime

random.seed(datetime.now())

C:\Users\hadit\AppData\Local\Temp\ipykernel_49396\3623220607.py:15: DeprecationWarning: Seeding based on hashing is deprecated
since Python 3.9 and will be removed in a subsequent version. The only 
supported seed types are: None, int, float, str, bytes, and bytearray.
  random.seed(datetime.now())


### Input format
The first line is of the form
```
num_agents,nego_est_o,nego_dl_o,tasks_est_o,tasks_dl_o
```
Here,
- `num_agents` is the number of agents
- `nego_est_o` is the outside earliest starting time of the negotiation
- `nego_dl_o` is the outside deadline of the negotiation
- `tasks_est_o` is the outside earliest starting time of the task execution
- `tasks_dl_o` is the outside deadline of the task execution

It is followed by `issue_num` lines, of which line $i$ (zero-indexed) of the input .csv file pertains to issue $i$, and is a tuple of the form `(agent, parent_issue, [pretasks], children_finish, issue_type, other_issue, pbs, reward, decommitment_penalty_rate, est_o, dl_o, duration)`, where

- `agent` is the agent ID of that agent which is supposed to do this task
- `parent_issue` of an incoming issue is **not** the same as that of its `other_issue`
- `[pretasks]` = Set of explicit pretasks (i.e., doesn't include parent-child relationships). '' if there is no pretask. Separated by ';' otherwise
- `children_finish` = 0 for "and", 1 for "or", '' for no chlidren. It may be 0 or 1 even if it has no children; however, '' indicates that we know for sure that there is no child.
- `issue_type` = 1 for local tasks (that are not incoming), 2 for outgoing tasks, 3 for incoming tasks
- `other_issue` is the issue ID of the issue linked to this issue (defined only when issue_type is outgoing or incoming)
- `pbs` is the base probability of success
- `reward` is the completion reward of the task, defined only for incoming issues
- `decommitment_penalty_rate` is the decommitment penalty rate, defined only for incoming issues. It is implicit that it is the same value for the corresponding outgoing issue. Meaningless for other local tasks.
- `est_o` is the outside earliest starting time of this task. The default is the overall `est_o`. For outgoing issues, it is undefined and taken to be that of the `other_issue`'s.
- `dl_o` is the outside deadline of this task. The default is the overall `dl_o`. For outgoing issues, it is undefined and taken to be that of the `other_issue`'s.
- `duration` is the number of timesteps needed to execute this task. Undefined for outgoing tasks.

### Defining classes to store the data

In [675]:
class issue_type(Enum):
    NONE = 0
    LOCAL = 1
    OUTGOING = 2
    INCOMING = 3


class issue:
    def __init__(self, issue_num: int, line: str, tasks_est_o, tasks_dl_o) -> None:
        self.ID = issue_num
        line = line.split(",")
        self.agent = int(line[0])
        self.parent = int(line[1])
        try:
            self.pretasks = list(map(int, line[2].split(";")))
        except:
            self.pretasks: list[int] = []
        self.posttasks: list[int] = []

        self.children_finish = int(line[3])
        self.issue_type = issue_type(int(line[4]))
        self.other_issue = int(line[5])
        self.pbs = float(line[6])
        if self.issue_type == issue_type.INCOMING:
            self.reward = float(line[7])
            self.decom_penalty_rate = float(line[8])
        else:
            self.reward = 0
            self.decom_penalty_rate = 0
        self.est_o = int(line[9])
        self.dl_o = int(line[10])
        self.duration = int(line[11])
        self.need_process_time = self.duration

        self.est = max(self.est_o, tasks_est_o)
        self.dl = min(self.dl_o, tasks_dl_o)

        self.eft = 0
        self.lst = 0

        self.negotiation_success: int = -1
        # -1 for unknown, 0 for failure, 1 for success

    def __str__(self) -> str:
        return f"agent:{self.agent} parent:{self.parent} pretasks:{self.pretasks} children_finish:{self.children_finish} issue_type:{self.issue_type.name} other_issue:{self.other_issue} pbs:{self.pbs} r:{self.reward} p:{self.decom_penalty_rate}"

In [676]:
class agent:
    class graph:
        def __init__(self) -> None:
            self.adj_list: dict[int, list[int]] = {}
            self.roots: list[int] = []

        def set(self, issue_graph, agent_) -> None:
            self.adj_list = {issue_ID: [] for issue_ID in agent_.issues}
            for issue_ID in agent_.issues:
                if issue_graph[issue_ID].parent == -1:
                    self.roots.append(issue_ID)
                    continue
                self.adj_list[issue_graph[issue_ID].parent].append(issue_ID)

        def print(self, agent_ID) -> None:
            print(f"Agent {agent_ID}:")
            print("Root(s): ", end="")
            print(*self.roots, sep=", ")
            for issue_ID in self.adj_list:
                print(f"{issue_ID} -> {self.adj_list[issue_ID]}")
            print()

        def dfs(self, issue_ID, visited, ans):
            visited[issue_ID] = True
            for child in self.adj_list[issue_ID]:
                if not visited[child]:
                    self.dfs(child, visited, ans)
            ans.append(issue_ID)

        def topological_sort(self) -> None:
            visited = {issue_ID: False for issue_ID in self.adj_list}
            ans: list[int] = []
            for issue_ID in self.adj_list:
                if not visited[issue_ID]:
                    self.dfs(issue_ID, visited, ans)
            self.toposorted_graph = ans

    def __init__(self, nego_dl_o) -> None:
        self.issues: list[int] = []

        # issues that are either outgoing or incoming
        self.negotiation_issues: list[int] = []

        # 0 for free, 1 for occupied. Length is the number of timesteps
        self.occupied: list[int] = [0 for _ in range(nego_dl_o + 1)]

        self.assignment: tuple[tuple[int, int, int]] = ()
        self.ordering: tuple[int] = ()
        self.best_value: int = float("-inf")

        self.problem_graph = self.graph()

        self.linear_schedule: list[tuple[int, int, int]] = []

    def __str__(self) -> str:
        return f"issues:{self.issues} negotiation_issues:{self.negotiation_issues} assignment:{self.assignment} ordering:{self.ordering} best_value:{self.best_value}"

### Load the data

In [677]:
input_file = os.path.join(os.getcwd(), "..", "data", "input.csv")

issue_graph: list[issue] = []
with open(input_file, "r") as f:
    num_agents, nego_est_o, nego_dl_o, tasks_est_o, tasks_dl_o = list(
        map(int, f.readline().split(","))
    )
    agents: list[agent] = [agent(nego_dl_o) for i in range(num_agents)]
    issue_num: int = 0
    for line in f:
        cur_issue = issue(issue_num, line, tasks_est_o, tasks_dl_o)
        issue_graph.append(cur_issue)
        agents[cur_issue.agent].issues.append(issue_num)
        issue_num += 1
    for i, issue_ in enumerate(issue_graph):
        # copying values from incoming issues to their corresponding outgoing issues
        if issue_.issue_type == issue_type.OUTGOING:
            issue_graph[i].decom_penalty_rate = issue_graph[
                issue_.other_issue
            ].decom_penalty_rate
            issue_graph[i].reward = issue_graph[issue_.other_issue].reward
            issue_graph[i].est_o = issue_graph[issue_.other_issue].est_o
            issue_graph[i].dl_o = issue_graph[issue_.other_issue].dl_o
            issue_graph[i].duration = issue_graph[issue_.other_issue].duration

print(f"{nego_est_o=}")
print(f"{nego_dl_o=}")
print(f"{num_agents=}")
print("Agents and their issues:")
for i, agent in enumerate(agents):
    print(f"Agent {i}: {agent.issues}")

print("\nIssues and their components:")
print(
    tabulate(
        [
            [
                i,
                issue_graph[i].agent,
                issue_graph[i].parent,
                issue_graph[i].pretasks,
                issue_graph[i].children_finish,
                issue_graph[i].issue_type.name,
                issue_graph[i].other_issue,
                issue_graph[i].pbs,
                issue_graph[i].reward,
                issue_graph[i].decom_penalty_rate,
                issue_graph[i].est_o,
                issue_graph[i].dl_o,
                issue_graph[i].duration,
            ]
            for i in range(len(issue_graph))
        ],
        headers=[
            "ID",
            "agent",
            "parent",
            "pretasks",
            "children_finish",
            "issue_type",
            "other_issue",
            "pbs",
            "reward",
            "decom_penalty_rate",
            "est_o",
            "dl_o",
            "duration",
        ],
    )
)

nego_est_o=0
nego_dl_o=10
num_agents=3
Agents and their issues:
Agent 0: [0, 1]
Agent 1: [2, 3, 4, 5, 6, 7, 8]
Agent 2: [9, 10, 11, 12]

Issues and their components:
  ID    agent    parent  pretasks      children_finish  issue_type      other_issue    pbs    reward    decom_penalty_rate    est_o    dl_o    duration
----  -------  --------  ----------  -----------------  ------------  -------------  -----  --------  --------------------  -------  ------  ----------
   0        0        -1  []                          0  OUTGOING                  2   0.98        10                   1          1      27           2
   1        0        -1  []                          0  OUTGOING                 10   0.89         9                   0.4        3      30           5
   2        1        -1  []                          0  INCOMING                  0   0.7         10                   1          1      27           2
   3        1         2  []                          0  LOCAL             

# Phase 1: Pre Negotiation

### Finding those issues that are going to be negotiated over

In [678]:
def find_negotiation_issues():
    print("Negotiation issues:")
    for i, agent in enumerate(agents):
        agent.negotiation_issues = [
            issueID
            for issueID in agent.issues
            if issue_graph[issueID].issue_type
            in [issue_type.OUTGOING, issue_type.INCOMING]
        ]
        print(f"Agent {i}: {agent.negotiation_issues}")


find_negotiation_issues()

Negotiation issues:
Agent 0: [0, 1]
Agent 1: [2, 6]
Agent 2: [9, 10]


### Complete search for the optimal (negotiation ordering, feature assignment tuple)

Helper functions to calculate flexibility, ps (probability of success), and EV (expected value):

In [679]:
def flexibility(est, dl, duration) -> float:
    """
    Returns the flexibility of the task in the range [0, 1]
    """
    return (dl - est - duration) / duration


def ps(pbs, est, dl, duration, c=0):
    """
    ps: probability of success
    """
    return 2 * pbs * (atan(flexibility(est, dl, duration))) / np.pi


def EV(
    assignment: tuple[tuple[int, int, int]],
    agent_ID: int,
) -> float:
    """
    Returns the estimated value of the assignment for the agent
    """
    ev: float = 0
    pbs_list = [
        issue_graph[issueID].pbs for issueID in agents[agent_ID].negotiation_issues
    ]
    ps_list = [
        ps(pbs, est, dl, dur) for (est, dl, dur), pbs in zip(assignment, pbs_list)
    ]
    issue_ID_list = [issueID for issueID in agents[agent_ID].negotiation_issues]
    for outcome in product([0, 1], repeat=len(assignment)):
        p = np.prod([ps if outcome[i] else 1 - ps for i, ps in enumerate(ps_list)])
        r = np.sum(
            [
                issue_graph[issueID].reward
                if outcome[i]
                else issue_graph[issueID].reward
                * issue_graph[issueID].decom_penalty_rate
                for i, issueID in zip(range(len(outcome)), issue_ID_list)
            ]
        )
        ev += p * r
    return ev


def is_valid_assignment(assignment) -> bool:
    """
    Returns true if the assignment is valid, false otherwise
    """
    for (est, dl, dur) in assignment:
        if est + dur > dl or est >= dl:
            return False

    def compare(a, b):
        # TODO: this may not be correct, how to find out if an assignment is valid?
        """
        Sorts by deadline first (lowest first), then est (lowest first), then duration (highest first)
        """
        if a[0] != b[0]:
            return a[0] - b[0]
        elif a[1] != b[1]:
            return a[1] - b[1]
        else:
            return b[2] - a[2]

    sorted_ass = list(assignment)
    sorted_ass.sort(key=cmp_to_key(compare))
    t = -1  # any negative number suffices since est >= 0 so t+1 >= 0
    for (est, dl, dur) in sorted_ass:
        t = max(t + 1, est) + dur
        if t > min(dl, nego_dl_o):
            return False

    return True


def find_ordering(assignment):
    """
    Returns a tuple of *indices* of the agents negotiation issues by inferring the ordering from the assignment
    """

    def compare(a, b):
        """
        Sorting by est and then deadline
        """
        return a[0] - b[0] if a[0] != b[0] else a[1] - b[1]

    return tuple((i[0] for i in sorted(enumerate(assignment), key=cmp_to_key(compare))))

The complete search itself:

In [680]:
def complete_search() -> None:
    """
    Finds the best assignment for each agent
    """
    # TODO: multithread this by making a thread for each agent
    for agent_ID, agent in enumerate(agents):
        best_value = float("-inf")
        best_assignment = None
        est_space = range(nego_est_o, nego_dl_o)
        dl_space = range(nego_est_o, nego_dl_o + 1)
        duration_space = range(1, nego_dl_o - nego_est_o)
        for assignment in product(
            product(est_space, dl_space, duration_space),
            repeat=len(agent.negotiation_issues),
        ):
            if is_valid_assignment(assignment):
                ev = EV(assignment, agent_ID)
                if ev > best_value:
                    best_value = ev
                    best_assignment = assignment
        ordering = find_ordering(assignment)
        agents[agent_ID].assignment = best_assignment
        agents[agent_ID].ordering = ordering
        agents[agent_ID].best_value = best_value

    # tabulate the output
    print(
        tabulate(
            [
                (
                    agent_ID,
                    agent.negotiation_issues,
                    agent.best_value,
                    agent.assignment,
                    agent.ordering,
                )
                for (agent_ID, agent) in enumerate(agents)
            ],
            headers=[
                "Agent ID",
                "Negotiation Issues",
                "Best EV",
                "Assignment",
                "Ordering",
            ],
        )
    )


complete_search()

  Agent ID  Negotiation Issues      Best EV  Assignment               Ordering
----------  --------------------  ---------  -----------------------  ----------
         0  [0, 1]                  18.0674  ((0, 7, 6), (0, 10, 1))  (0, 1)
         1  [2, 6]                  29.5     ((0, 7, 4), (0, 1, 1))   (0, 1)
         2  [9, 10]                 27.969   ((0, 1, 1), (0, 10, 1))  (0, 1)


# Phase 2: Negotiations

In [681]:
# Following the earliest-finish-time policy
def run_negotiations() -> None:
    for agent_ID, agent in enumerate(agents):
        ordered_issues = [agent.negotiation_issues[i] for i in agent.ordering]
        ordered_assignment = [agent.assignment[i] for i in agent.ordering]
        for issue_ID in ordered_issues:
            if issue_graph[issue_ID].negotiation_success != -1:
                continue
            other_issue_ID = issue_graph[issue_ID].other_issue
            other_agent_ID = issue_graph[other_issue_ID].agent
            time_steps: list[int] = []

            nego_duration = ordered_assignment[ordered_issues.index(issue_ID)][2]
            for i in range(len(agent.occupied)):
                if len(time_steps) == nego_duration:
                    break
                if not agent.occupied[i] and not agents[other_agent_ID].occupied[i]:
                    time_steps.append(i)

            if len(time_steps) < nego_duration:
                # negotiation failed
                issue_graph[issue_ID].negotiation_success = 0
                issue_graph[other_issue_ID].negotiation_success = 0
                print(
                    f"Negotiation failed for agents {agent_ID} and {other_agent_ID} over issue {issue_ID}"
                )
                continue

            # negotiation has succeeded:
            print(
                f"Negotiation succeeded for agents {agent_ID} and {other_agent_ID} over issue {issue_ID}. Time steps: {time_steps} (duration: {nego_duration})"
            )
            issue_graph[issue_ID].negotiation_success = 1
            issue_graph[other_issue_ID].negotiation_success = 1

    # set the features of the issue execution for phase 3
    for issue_ID, issue in enumerate(issue_graph):
        if (
            issue.issue_type in [issue_type.INCOMING, issue_type.OUTGOING]
            and issue.other_issue > issue_ID
        ):
            free_time = (
                issue_graph[issue_ID].dl_o
                - issue_graph[issue_ID].est_o
                - issue_graph[issue_ID].duration
            )
            issue_graph[issue_ID].est = issue_graph[issue_ID].est_o + random.randint(
                0, int(free_time / 2)
            )
            issue_graph[issue_ID].dl = issue_graph[issue_ID].dl_o - random.randint(
                0, free_time - int(free_time / 2)
            )
            issue_graph[issue.other_issue].est = issue_graph[issue_ID].est
            issue_graph[issue.other_issue].dl = issue_graph[issue_ID].dl

    # tabulate issue, negotiation success, est, dl, duration
    negotiation_success_in_words = {1: "Success", 0: "Failure", -1: "NA"}
    print(
        tabulate(
            [
                (
                    issue_ID,
                    issue.negotiation_success,
                    issue.est,
                    issue.dl,
                    issue.duration,
                )
                for issue_ID, issue in enumerate(issue_graph)
            ],
            headers=["Issue ID", "Negotiation Success", "est", "dl", "duration"],
        )
    )


run_negotiations()

Negotiation succeeded for agents 0 and 1 over issue 0. Time steps: [0, 1, 2, 3, 4, 5] (duration: 6)
Negotiation succeeded for agents 0 and 2 over issue 1. Time steps: [0] (duration: 1)
Negotiation succeeded for agents 1 and 2 over issue 6. Time steps: [0] (duration: 1)
  Issue ID    Negotiation Success    est    dl    duration
----------  ---------------------  -----  ----  ----------
         0                      1      9    24           2
         1                      1     14    21           5
         2                      1      9    24           2
         3                     -1      1    26           1
         4                     -1      1    25           1
         5                     -1      2    30           1
         6                      1      4    20           5
         7                     -1      2    27           1
         8                     -1      9    30           1
         9                      1      4    20           5
        10            

# Phase 3: Scheduling Tasks

### Convert input file into dependency graphs

In [682]:
def set_problem_graphs() -> None:
    print("PROBLEM GRAPHS (adjacency lists):")
    for agent_ID, agent in enumerate(agents):
        agent.problem_graph.set(issue_graph, agent)
        agent.problem_graph.print(agent_ID)


set_problem_graphs()

PROBLEM GRAPHS (adjacency lists):
Agent 0:
Root(s): 0, 1
0 -> []
1 -> []

Agent 1:
Root(s): 2
2 -> [3]
3 -> [4, 5]
4 -> [6, 7, 8]
5 -> []
6 -> []
7 -> []
8 -> []

Agent 2:
Root(s): 9, 10
9 -> [11]
10 -> [12]
11 -> []
12 -> []



### Topologically sort to obtain partial-ordered schedules

In [683]:
def topological_sort() -> None:
    for agent_ID, agent in enumerate(agents):
        agent.problem_graph.topological_sort()
        print(
            f"Topological sort for agent {agent_ID}: {agent.problem_graph.toposorted_graph}"
        )


def remove_failed_issues() -> None:
    # TODO
    for agent_ID, agent in enumerate(agents):
        visited = {issue_ID: False for issue_ID in agent.problem_graph.toposorted_graph}


topological_sort()
remove_failed_issues()

Topological sort for agent 0: [0, 1]
Topological sort for agent 1: [6, 7, 8, 4, 5, 3, 2]
Topological sort for agent 2: [11, 9, 12, 10]


### Propagate earliest start time and deadline

In [684]:
def eft(issue_ID_list: list[int]) -> int:
    """
    Returns the earliest finish time of the given list of issues
    """
    return max(
        [
            max(
                issue_graph[issue_ID].est + issue_graph[issue_ID].duration,
                issue_graph[issue_ID].eft,
            )
            for issue_ID in issue_ID_list
        ],
        default=0,
    )


def lst(issue_ID_list: list[int]) -> int:
    """
    Returns the latest start time of the given list of issues
    """
    return min(
        [
            issue_graph[issue_ID].dl - issue_graph[issue_ID].duration
            for issue_ID in issue_ID_list
        ],
        default=tasks_dl_o,
    )


def update_est() -> None:
    """
    Updates the est of each issue in the problem graph
    """
    for agent_ID, agent in enumerate(agents):
        S = copy(agent.problem_graph.toposorted_graph)
        while S:
            for issue_ID in S:
                if not any(
                    pretask in S for pretask in issue_graph[issue_ID].pretasks
                ) and not any(
                    child in S for child in agent.problem_graph.adj_list[issue_ID]
                ):
                    issue_graph[issue_ID].est = max(
                        eft(
                            issue_graph[issue_ID].pretasks
                            + agent.problem_graph.adj_list[issue_ID]
                        ),
                        issue_graph[issue_ID].est_o,
                    )
                    issue_graph[issue_ID].eft = (
                        issue_graph[issue_ID].est + issue_graph[issue_ID].duration
                    )
                    S.remove(issue_ID)
                    break


def set_posttasks() -> None:
    """
    Infers the posttasks of each issue from the pretasks given in the input
    """
    for issue_ID, issue in enumerate(issue_graph):
        for pretask in issue.pretasks:
            issue_graph[pretask].posttasks.append(issue_ID)


def update_dl() -> None:
    for agent_ID, agent in enumerate(agents):
        S = copy(agent.problem_graph.toposorted_graph)
        while S:
            for issue_ID in S:
                if not any(
                    posttask in S for posttask in issue_graph[issue_ID].posttasks
                ) and not any(
                    child in S for child in agent.problem_graph.adj_list[issue_ID]
                ):
                    issue_graph[issue_ID].dl = min(
                        lst(
                            issue_graph[issue_ID].posttasks
                            + agent.problem_graph.adj_list[issue_ID]
                        ),
                        issue_graph[issue_ID].dl_o,
                    )
                    issue_graph[issue_ID].lst = (
                        issue_graph[issue_ID].dl - issue_graph[issue_ID].duration
                    )
                    S.remove(issue_ID)
                    break


def print_est_dl() -> None:
    print(
        tabulate(
            [
                (issue_ID, issue.est, issue.dl)
                for issue_ID, issue in enumerate(issue_graph)
            ],
            headers=["Issue ID", "est", "dl"],
        )
    )


update_est()
set_posttasks()
update_dl()
print_est_dl()

  Issue ID    est    dl
----------  -----  ----
         0      1    27
         1      3    30
         2     13    19
         3     12    20
         4     10    21
         5     11    30
         6      4    26
         7      2    27
         8      9    30
         9     11    19
        10     16    20
        11      4    26
        12      7    29


### Generate linear schedule

In [685]:
def feasible_schedule(agent_ID, agent):
    """
    Returns a feasible linear schedule from the given partial order schedule if it is valid, else returns None
    """
    V = copy(agent.issues)
    V.sort(key=lambda issue_ID: issue_graph[issue_ID].est)
    st_i = max(tasks_est_o, issue_graph[V[0]].est)
    st_ii = get_next_check_point(V, st_i)
    V.sort(key=lambda issue_ID: issue_graph[issue_ID].dl)
    while V:
        in_process = False
        task_ready = False
        for issue_ID in V:
            if is_ready(V, issue_ID, agent_ID):
                task_ready = True
            if issue_graph[issue_ID].issue_type == issue_type.OUTGOING:
                V.remove(issue_ID)
                in_process = True
        for issue_ID in V:
            t = issue_graph[issue_ID]
            if (
                is_ready(V, issue_ID, agent_ID)
                and t.issue_type in [issue_type.LOCAL, issue_type.INCOMING]
                and t.est <= st_i
            ):
                if agent_ID == 0:
                    print("here")
                if st_i + t.need_process_time > t.dl:
                    print(f"Agent {agent_ID} is too late with issue {issue_ID}")
                    return None
                if st_ii - st_i < t.need_process_time:
                    agent.linear_schedule.append((st_i, st_ii, issue_ID, "a"))
                    t.need_process_time = t.need_process_time - (st_ii - st_i)
                    st_i = st_ii
                    st_ii = get_next_check_point(V, st_i)
                    in_process = True
                if st_ii - st_i >= t.need_process_time:
                    agent.linear_schedule.append(
                        (st_i, st_i + t.need_process_time, issue_ID, "b")
                    )
                    V.remove(issue_ID)
                    st_i += t.need_process_time
                    st_ii = get_next_check_point(V, st_i)
                    t.need_process_time = 0
                    in_process = True
                V.sort(key=lambda issue_ID: issue_graph[issue_ID].dl)
                break
        if task_ready and not in_process:
            agent.linear_schedule.append((st_i, st_ii, -1, "c"))
            st_i = st_ii
            st_ii = get_next_check_point(V, st_i)
            V.sort(key=lambda issue_ID: issue_graph[issue_ID].dl)
        elif not task_ready:
            print(f"No progress for the schedule of agent {agent_ID}!")
            return None
    return agent.linear_schedule


def get_next_check_point(issue_ID_list: list[int], st):
    min_dl = float("inf")
    for issue_ID in issue_ID_list:
        t = issue_graph[issue_ID]
        if t.est <= st:
            if t.dl < min_dl and t.dl > st:
                min_dl = t.dl
        else:
            return min(min_dl, t.est)
    return min_dl


def is_ready(V: list[int], issue_ID: int, agent_ID: int) -> bool:
    return not (
        any([pretask in V for pretask in issue_graph[issue_ID].pretasks])
        or any(
            [child in V for child in agents[agent_ID].problem_graph.adj_list[issue_ID]]
        )
    )


for agent_ID, agent in enumerate(agents):
    print(f"Agent {agent_ID}: ")
    if feasible_schedule(agent_ID, agent) is not None:
        # print(f'{agent.linear_schedule}')
        print(
            tabulate(
                [
                    (st_i, st_ii, issue_ID, issue_graph[issue_ID].issue_type.name)
                    for st_i, st_ii, issue_ID, _ in agent.linear_schedule
                ],
                headers=["st_i", "st_ii", "issue_ID", "issue_type"],
            )
        )
    else:
        print("No feasible schedule found!")

    print()

Agent 0: 
st_i    st_ii    issue_ID    issue_type
------  -------  ----------  ------------

Agent 1: 
  st_i    st_ii    issue_ID  issue_type
------  -------  ----------  -------------------
     2        3           7  issue_type.LOCAL
     3       13          -1  issue_type.LOCAL
    13       14           8  issue_type.LOCAL
    14       15           4  issue_type.LOCAL
    15       16           5  issue_type.LOCAL
    16       17           3  issue_type.LOCAL
    17       19           2  issue_type.INCOMING

Agent 2: 
Agent 2 is too late with issue 10
No feasible schedule found!

